# Chicago Crime Data Project: Part 1

**Steps:**

1. Use data from the Chicago Data Portal: Crimes 2001 to Present, which includes type of crime, date/time, lat/long, District/ward, arrests, etc.
    - https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2

2. Use this data processing helper notebook to process large Chicago crime data .csv into several smaller .csv.gz files for analysis.
    - https://github.com/coding-dojo-data-science/preparing-chicago-crime-data

3. Answer several questions about the data. Questions:
    - (1) Comparing Police Districts: 
        - (A) Which district has the most crimes? 
        - (B) Which has the least?
    - (2) Crimes Across the Years:
        - (A) Is the total number of crimes increasing or decreasing across the years?
        - (B) Are there any individual crimes that are doing the opposite (i.e., decreasing when orverall crime is increasing or vice-versa)?
    - (3) Comparing AM vs. PM Rush Hour:
        - (A) Are crimes more common during AM rush hour or PM rush hour? Consider 7am-10am and 4pm-7pm as rush hours.
        - (B) What are the top 5 most common crimes during AM rush hour? What are the top 5 most common crimes during PM rush hour?
        - (C) Are MOtor Vehicle Thefts more common during AM rush hour or PM rush hour?
    - (4) Comparing Months:
        - (A) What months have the most crime? What months have the least?
        - (B) Are there any individual crimes that do not follow this pattern? If so, which crimes?
    - (5) Comparing Holidays:
        - (A) Are there any holidays that show an increase in the number of crimes?
        - (B) Are there any holidays that show a decrease in the number of crimes?

# 1 Imports

In [2]:
# imports
import pandas as pd

# 2 Data Preparation